In [16]:
file_data = open('day16.data', 'r').read().splitlines()

import re
from typing import Dict, Tuple
from collections import namedtuple

Valve = namedtuple('Valve', 'flow_rate connected_valves')

valve_map: Dict[str, Valve] = {}

for line in file_data:
    valve, flow_rate, to_valves = re.findall(r'Valve (\w+) has flow rate=(\d+); tunnels? leads? to valves? (.*)', line)[0]
    valve_map[valve] = Valve(int(flow_rate), tuple(to_valves.split(', ')))

In [17]:
from itertools import permutations

valve_to_valve_distance: Dict[Tuple[str, str], int] = dict()

important_valves = ('AA',) + tuple(v for v in valve_map if valve_map[v].flow_rate != 0)

for valve_from, valve_to in permutations(important_valves, 2):
    ways = list([v] for v in valve_map[valve_from].connected_valves)

    while ways:
        current_way = ways.pop(0)
        
        if current_way[-1] == valve_to:
            valve_to_valve_distance[(valve_from, valve_to)] = len(current_way)
            break

        last_valve = current_way[-1]
        connected_valves = valve_map[last_valve].connected_valves

        ways += [current_way + [v] for v in connected_valves]

In [18]:
max_total_flow = 0

State = namedtuple('State', 'way closed_valves minute total_flow')
states = [State(('AA',), set(important_valves) - set(('AA',)), 1, 0)]

while states:
    state = states.pop(0)

    for next_valve in state.closed_valves:
        minute = state.minute + valve_to_valve_distance[(state.way[-1], next_valve)] + 1
        if minute >= 30:
            continue
        total_flow = state.total_flow + (31 - minute) * valve_map[next_valve].flow_rate
        next_state = State(state.way + (next_valve,), state.closed_valves - set((next_valve,)), minute, total_flow)
        if total_flow > max_total_flow:
            max_total_flow = total_flow
            max_state = next_state

        states.append(next_state)

print(max_state)


State(way=('AA', 'VK', 'GQ', 'YL', 'YA', 'AZ', 'EY', 'JT'), closed_valves={'RK', 'ES', 'UI', 'TR', 'XN', 'AT', 'OC', 'WZ'}, minute=25, total_flow=1828)
